In [1]:
import os
%pwd

'g:\\PROJECT\\Text-Summarizer\\research'

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories


In [5]:
class ConfigurationManager:

    def __init__(self,
                config_path=CONFIG_FILE_PATH,
                params_path=PARAMS_FILE_PATH):
        self.config= read_yaml(config_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config= self.config.model_evaluation
        model_evaluation_config= ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_path= config.model_path,
            tokenizer_path= config.tokenizer_path,
            metric_file_name= config.metric_file_name
        )
        return model_evaluation_config

        

In [12]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk
from tqdm import tqdm


from transformers import TrainingArguments
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate



In [18]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config= config
        m=os.path.dirname(self.config.metric_file_name)
        create_directories([m])

    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                    column_text="article", 
                                    column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            # FIX 1: Add error handling for tokenization
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            # FIX 2: Check for out-of-vocabulary tokens
            max_token_id = inputs["input_ids"].max().item()
            vocab_size = model.config.vocab_size
            
            if max_token_id >= vocab_size:
                print(f"Warning: Token ID {max_token_id} exceeds vocab size {vocab_size}")
                # Skip this batch or handle appropriately
                continue
        
            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device), 
                length_penalty=0.8, 
                num_beams=8, 
                max_length=128
            )
            
            # Decode the generated texts
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        # Compute and return the ROUGE scores
        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path, use_fast=True)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        # FIX 3: Load tokenizer from the same path as model (use trained tokenizer)
        #tokenizer = tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum", use_fast=True)
        #model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print("\n=== Diagnostic Information ===")
        print(f"Model vocab size: {model_pegasus.config.vocab_size}")
        print(f"Tokenizer vocab size: {len(tokenizer)}")

        # Test tokenization on a sample
        sample_text = dataset_samsum_pt['test'][0]['dialogue']
        test_tokens = tokenizer(sample_text, return_tensors="pt")
        print(f"Max token ID in sample: {test_tokens['input_ids'].max().item()}")
        print(f"Sample token IDs: {test_tokens['input_ids'][0][:20]}") 
        # FIX 4: Verify tokenizer and model compatibility
        print(f"Model vocab size: {model_pegasus.config.vocab_size}")
        print(f"Tokenizer vocab size: {len(tokenizer)}")
        
        # Loading data 
        

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], 
            rouge_metric, 
            model_pegasus, 
            tokenizer, 
            batch_size=2, 
            column_text='dialogue', 
            column_summary='summary'
        )

        # Extract scores
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [19]:
from datasets import load_from_disk

dataset_samsum_pt = load_from_disk("artifacts/data_transformation/transformed_dataset")

print(dataset_samsum_pt["test"][0])


{'id': '13862856', 'dialogue': "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye", 'summary': "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.", 'input_ids': [12636, 151, 10532, 108, 171, 119, 133, 17379, 131, 116, 344, 152, 12195, 151, 1054, 55700, 553, 12636, 151, 110, 105, 12014, 940, 56220, 2314, 12195, 151, 9752, 108, 137, 131, 144, 258, 126, 107, 12195, 151, 5989, 9915, 12195, 151, 285, 568, 215, 289, 166, 145, 195, 134, 109, 1669, 424, 12636, 151, 125, 272, 131, 144, 235, 342, 210, 12636, 151, 110, 105, 12014, 940, 56220, 2314, 12195, 151, 1414, 131, 144, 129, 9279, 108, 178, 1

In [20]:
config=ConfigurationManager()
model_evaluation_config=config.get_model_evaluation_config()
model_evaluation=ModelEvaluation(config=model_evaluation_config)
model_evaluation.evaluate()


[2025-12-11 00:52:22,963: INFO: common: yaml file config\config.yaml loaded successfully:]
[2025-12-11 00:52:22,985: INFO: common: yaml file params.yaml loaded successfully:]
[2025-12-11 00:52:22,989: INFO: common: created directory atartifacts:]
[2025-12-11 00:52:22,993: INFO: common: created directory atartifacts/model_evaluation:]



=== Diagnostic Information ===
Model vocab size: 96103
Tokenizer vocab size: 96103
Max token ID in sample: 58779
Sample token IDs: tensor([12636,   151, 10532,   108,   171,   119,   133, 17379,   131,   116,
          344,   152, 12195,   151,  1054, 55700,   553, 12636,   151,   110])
Model vocab size: 96103
Tokenizer vocab size: 96103


100%|██████████| 5/5 [06:26<00:00, 77.24s/it]


[2025-12-11 00:58:54,213: INFO: rouge_scorer: Using default tokenizer.:]


In [42]:
t = AutoTokenizer.from_pretrained("google/pegasus-xsum")
print("Correct vocab:", t.vocab_size)



Correct vocab: 96103


In [44]:
from transformers import AutoTokenizer

t = AutoTokenizer.from_pretrained("google/pegasus-xsum", force_download=True)
print("Vocab:", t.vocab_size)


g:\PROJECT\Text-Summarizer\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--google--pegasus-xsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Vocab: 96103
